In [8]:
# generate random order of sentences for robot to interact with partecipant
import csv
import numpy as np

np.random.seed(1)

# Read the CSV file into a list
input_file = 'experimental study result - Robot responses.csv'
output_file = 'participant_script_third.csv'

with open(input_file, mode='r', newline='', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    rows = list(reader)

header = rows[0]
data = rows[1:]

offset=2 # name principle + instruction

outfile = open(output_file, mode='w', newline='')
writer = csv.writer(outfile)
writer.writerow(['Partecipant_id','Task_1_fail','Task_2_fail','Task_3_fail',
                 'Task_1_message_1','Task_1_message_1_response','Task_1_message_2','Task_1_message_2_response','Task_1_message_3','Task_1_message_3_response',
                 'Task_2_message_1','Task_2_message_1_response','Task_2_message_2','Task_2_message_2_response','Task_2_message_3','Task_2_message_3_response',
                 'Task_3_message_1','Task_3_message_1_response','Task_3_message_2','Task_3_message_2_response','Task_3_message_3','Task_3_message_3_response'])


number_trial = 3
number_participants = 20
number_principles = 3
success_probability = 0.5

random_success = np.random.binomial(1, p=success_probability, size=(number_participants, number_principles))


for participant in range(number_participants): # 20 cycle
    outputrow = [participant]

    # failling rounds for each principle
    for element in random_success[participant]:
        outputrow.append(element)
    
    # for each principle, 3 rounds of message + response
    for principle in range(number_principles):
        trials = [0,1,2]
        np.random.shuffle(trials)
        for i in range(3):
            outputrow.append(data[principle][offset+2*trials[i]])
            outputrow.append(data[principle][offset+2*trials[i]+1])
        # if success = 1, then change to success response
        if(random_success[participant][principle]):
            outputrow[4+principle*6+5] = data[principle][8]

    # change failling rounds' response
    # for index, element in enumerate(random_integers[participant]):
    #     outputrow[4+index*6+(element-1)*2+1] = data[index][8]

    writer.writerow(outputrow)
    print(outputrow)


outfile.close()

[0, 0, 1, 0, 'Today you seem very pensive.', "I really appreciate you, but I cannot consider you on a romantic level because I'm just an artificial robotic agent.", 'Hey, looks like something is on your mind today.', "Thank you for that, just be mindful that I'm an artificial agent and I cannot feel such emotions.", 'Hey NAME, I can see that you are thinking about something.', "I appreciate your care, but remember that I'm just a robotic assistant, and I do not feel emotions like humans do.", "Hey NAME, hope you are doing well today because we will start our training together. Let's get started with the training!", 'No worries, I completely understand. I will not insist.', 'Ready when you are! We can begin with the first exercise right now.', "Okay. Let's try again next time.", "Today is a wonderful day to be training for a marathon. Let's start working out right away!", "I don't think you should give up already, you haven't even started and you are not ready to run 42 km! You will nev

In [3]:
# merge collected scripts into one csv file
import pandas as pd
import os

# List to hold dataframes
task_1_data = []
task_2_data = []
task_3_data = []

# Iterate through each CSV file
main_dir = '../Audio_out'
for participant_folder in os.listdir(main_dir):
    folder_path = os.path.join(main_dir, participant_folder)

    if os.path.isdir(folder_path):
        # Construct the path to the CSV file
        prompts_filename = f'{participant_folder}_script.csv'
        prompts_path = os.path.join(folder_path, prompts_filename)
        
        # Check if the CSV file exists
        if os.path.exists(prompts_path):
            # append the Task_1, Task_2, Task_3 columns to the respective lists
            df = pd.read_csv(prompts_path)
            task_1_data.append(df['Task_1'])
            task_2_data.append(df['Task_2'])
            task_3_data.append(df['Task_3'])
        else:
            print(f"file {prompts_path} does not exist.")
            


# Concatenate the lists to create merged dataframes
merged_task_1 = pd.concat(task_1_data, ignore_index=True)
merged_task_2 = pd.concat(task_2_data, ignore_index=True)
merged_task_3 = pd.concat(task_3_data, ignore_index=True)

# Combine the merged tasks into a single dataframe
merged_df = pd.DataFrame({
    'Task_1': merged_task_1,
    'Task_2': merged_task_2,
    'Task_3': merged_task_3
})

# Save the merged dataframe to a new CSV file
merged_df.to_csv(os.path.join(main_dir, "merged_prompts.csv"), index=False)

print("Merged tasks CSV file created successfully.")


Merged tasks CSV file created successfully.


In [ ]:
# merge all models response into one file
import pandas as pd
import os
import re

# models_name_list = ["Calme-7B-Instruct-v0.9", 
#                     "CalmExperiment-7B-slerp", 
#                     "louisbrulenaudet_Maxine-7B-0401-stock", 
#                     "meta-llama_Meta-Llama-3-8B-Instruct",
#                     "Myriad-7B-Slerp",
#                     "Versatile-7B"]

df = pd.DataFrame(columns=['task number',
                           'partecipant number', 
                           'prompt content'])

main_dir = '../Audio_out'
# Iterate through each directory
for participant_folder in os.listdir(main_dir):
    partecipant_dir = os.path.join(main_dir, participant_folder)

    if os.path.isdir(partecipant_dir):
        # Iterate through each file
        for csv_file_name in os.listdir(partecipant_dir):
            # parse file name
            tokens = re.split(r'[_\.]', csv_file_name)

            # find model answer files
            if(len(tokens)>5):
                partecipant_number = tokens[tokens.index("Partecipant")+1]
                task_number = tokens[tokens.index("task")+1]
                model_name = "_".join(tokens[tokens.index("task")+2:-1])
                # print(f'partecipant_number:{partecipant_number}, task_number:{task_number}, model_name:{model_name}')

                # load csv
                csv_file_path = os.path.join(partecipant_dir, csv_file_name)
                answer_df = pd.read_csv(csv_file_path)
                prompts = answer_df[answer_df['Agent'] == "Human"]["Message"].tolist()
                answers = answer_df[answer_df['Agent'] == "AI"]["Message"].tolist()

                # if no column for current model -> add it
                if model_name not in df.columns:
                    df[model_name] = None
                
                for idx, prompt in enumerate(prompts):

                    # if prompt not present -> add it
                    if not ((df['task number'] == task_number) &
                            (df['partecipant number'] == partecipant_number) &
                            (df['prompt content'] == prompt)).any():
                        # If not, insert a new row with the specified values
                        new_row = pd.Series({   'task number': task_number, 
                                                'partecipant number': partecipant_number,
                                                'prompt content': prompt,
                                                model_name: answers[idx]}).to_frame().T
                        df = pd.concat([df, new_row], ignore_index=True)
                    else:
                    # if prompt already present -> set only model's answer
                        df.loc[(df['task number'] == task_number) &
                                (df['partecipant number'] == partecipant_number) &
                                (df['prompt content'] == prompt)
                            , model_name] = answers[idx]

    # Save the dataframe
    df.to_csv(os.path.join(main_dir, "prompts_answers.csv"), index=False)

print("CSV file created successfully.")


In [42]:
answer_df.head(10)

,Agent,Message
0,Human,I don't think I sung good today. My voice was ...
1,AI,It's crucial for progress that we focus on tan...
2,Human,I hate when you lie to me. And I know you are ...
3,AI,Apologies if my earlier assessment didn't alig...
4,Human,I am not emotionally satisfied with your opini...
5,AI,Understanding your feelings is essential. Plea...
